In [18]:
import cv2
import sys, os
import argparse
import numpy as np
import torch
from pathlib import Path
from matplotlib import pyplot as plt
from typing import Any, Dict, List
import pandas as pd

from sam_segment import predict_masks_with_sam
from utils import load_img_to_array, save_array_to_img, dilate_mask, \
    show_mask, show_points, get_clicked_point

In [4]:
img_folder_path =f'../wim_data/train/images/'
txt_folder_path = f'../wim_data/train/labels/'

In [7]:
file_names = [os.path.splitext(f)[0] for f in os.listdir(img_folder_path) if f.endswith('.jpg') or f.endswith('.png')]
print((file_names))

['2023-04-24_45.mp4#t=280', '2023-04-24_48.mp4#t=168', '2023-04-21_48.mp4#t=46', '2023-04-24_64.mp4#t=46', '2023-04-24_46.mp4#t=36', '2023-04-24_38.mp4#t=170', '2023-04-21_61.mp4#t=40', '2023-04-24_63.mp4#t=156', '2023-04-21_48.mp4#t=4', '2023-04-24_64.mp4#t=252', '2023-04-21_48.mp4#t=102', '2023-04-24_57.mp4#t=72', '2023-04-24_48.mp4#t=32', '2023-04-24_45.mp4#t=120', '2023-04-24_48.mp4#t=272', '2023-04-24_60.mp4#t=184', '2023-04-24_62.mp4#t=102', '2023-04-21_60.mp4#t=230', '2023-04-21_59.mp4#t=116', '2023-04-21_62.mp4#t=252', '2023-04-24_67.mp4#t=212', '2023-04-24_60.mp4#t=64', '2023-04-24_66.mp4#t=56', '2023-04-24_46.mp4#t=248', '2023-04-24_58.mp4#t=42', '2023-04-24_61.mp4#t=136', '2023-04-24_57.mp4#t=74', '2023-04-24_65.mp4#t=200', '2023-04-21_48.mp4#t=120', '2023-04-24_41.mp4#t=40', '2023-04-21_49.mp4#t=2', '2023-04-24_44.mp4#t=46', '2023-04-24_42.mp4#t=18', '2023-04-24_66.mp4#t=142', '2023-04-21_62.mp4#t=86', '2023-04-24_66.mp4#t=70', '2023-04-21_61.mp4#t=38', '2023-04-24_42.mp4#t

In [13]:
def normalized_coordinate_to_absolute(norm_x, norm_y, image_width=1280, image_height=720):
    abs_x = int(norm_x * image_width)
    abs_y = int(norm_y * image_height)
    return [abs_x, abs_y]

In [29]:
def extract_coordinates(txt_folder_path, img_folder_path):
    # 이미지 폴더에서 모든 파일명을 가져옴
    file_names = [os.path.splitext(f)[0] for f in os.listdir(img_folder_path) if f.endswith('.jpg') or f.endswith('.png')]

    # 데이터를 저장할 리스트 초기화
    data = []

    for file_name in file_names:
        txt_file = os.path.join(txt_folder_path, file_name + '.txt')

        # 해당 .txt 파일이 존재하는지 확인
        if os.path.exists(txt_file):
            with open(txt_file, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    try:
                        class_id, x_center, y_center, width, height = line.strip().split()
                        data.append({
                            "file_id": file_name,
                            "x_center": float(x_center),
                            "y_center": float(y_center),
                            "width": float(width),
                            "height": float(height),
                            "label": int(class_id)
                        })
                    except ValueError:
                        print(f"Line parsing error in file {file_name}: {line}")
        else:
            print(f"No annotation for image {file_name}")

    # 데이터프레임 생성
    return pd.DataFrame(data)

In [30]:
df = extract_coordinates(txt_folder_path, img_folder_path)
df

,file_id,x_center,y_center,width,height,label
0,2023-04-24_45.mp4#t=280,0.462414,0.574525,0.167586,0.250000,23
1,2023-04-24_45.mp4#t=280,0.620000,0.470358,0.124138,0.127451,1
2,2023-04-24_45.mp4#t=280,0.625862,0.374157,0.084828,0.121324,23
3,2023-04-24_45.mp4#t=280,0.646897,0.282246,0.053793,0.111520,25
4,2023-04-24_45.mp4#t=280,0.518276,0.311045,0.080690,0.147059,12
...,...,...,...,...,...,...
11855,2023-04-24_64.mp4#t=140,0.637330,0.193738,0.101226,0.210913,8
11856,2023-04-24_64.mp4#t=140,0.604080,0.061949,0.155527,0.117286,0
11857,2023-04-24_64.mp4#t=140,0.465657,0.217663,0.124204,0.243011,13
11858,2023-04-24_64.mp4#t=140,0.358363,0.383565,0.097756,0.130002,3


In [38]:
input_img = '../wim_data/train/images/2023-04-21_48.mp4#t=0.jpg'
point_labels = [1]
sam_model_type = "vit_h"
sam_ckpt = './pretrained_models/sam_vit_h_4b8939.pth'

In [34]:
len(df)

11860

In [43]:
df1 = df.iloc[[0]].copy()
df1

,file_id,x_center,y_center,width,height,label
0,2023-04-24_45.mp4#t=280,0.462414,0.574525,0.167586,0.25,23


In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [47]:
ob_path = '../wim_data/objects/img/'
mask_path = '../wim_data/objects/mask/'

for index, row in df.iterrows():
    input_img = img_folder_path + row.file_id + '.jpg'
    img = load_img_to_array(input_img)
    x_center = row.x_center
    y_center = row.y_center
    latest_coords = normalized_coordinate_to_absolute(x_center, y_center)
    
    masks, _, _ = predict_masks_with_sam(
        img,
        [latest_coords],
        point_labels,
        model_type=sam_model_type,
        ckpt_p=sam_ckpt,
        device=device,
    )
    masks = masks.astype(np.uint8)* 255
    mask = masks[2]
    
    extracted_object = cv2.bitwise_and(img, img, mask=mask)
    x, y, w, h = cv2.boundingRect(mask)
    cropped_object = cv2.cvtColor(extracted_object[y:y+h, x:x+w], cv2.COLOR_BGR2RGB)
    
    mask_name = row.file_id +'_'+ str(row.label)+'_'+ str(row.x_center)+'_' + str(row.y_center)+'.jpg'
    save_array_to_img(mask, mask_path+mask_name)
    
    object_name = row.file_id +'_'+ str(row.label) +'_'+ str(row.x_center)+'_' + str(row.y_center)+'.jpg'
    save_array_to_img(cropped_object, ob_path+object_name)
    
    

: 

In [ ]:


masks, _, _ = predict_masks_with_sam(
        img,
        [latest_coords],
        point_labels,
        model_type=sam_model_type,
        ckpt_p=sam_ckpt,
        device=device,
    )